In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import torch
import numpy as np
import pandas as pd
import random 
#import sklearn
print("python: ",sys.version)
print("pytorch: ", torch.__version__)
print("numpy : ", np.__version__)
print("pandas : ", pd.__version__)
#print("sklearn : ", sklearn.__version__)

python:  3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
pytorch:  1.11.0+cu113
numpy :  1.21.6
pandas :  1.3.5


In [ ]:
%cd drive/MyDrive/Duolingo-Knowledge-Tracing

/content/drive/MyDrive/Duolingo-Knowledge-Tracing


In [ ]:
import SEKTInference

## 실시간으로 가장 높은 성취도파급효과 불러오는 문장 추천하기

실시간으로 넣을 문장 목록들 준비하기 (단어 + 품사 필수)

In [ ]:
df = pd.read_csv('data/train_data.csv')

In [ ]:
sen_list = []
pos_list = []

In [ ]:
# 약1분소요
sen = []
pos = []
for i in range(len(df)):
  if(i > 0 and df['split'][i] != df['split'][i-1]):
    sen_list.append(sen)
    pos_list.append(pos)
    sen = []
    pos = []

  sen.append(df['word'][i])
  pos.append(df['pos'][i])

In [ ]:
sentence_examples = []
part_of_speech = []

In [ ]:
save = []
for idx,pos in enumerate(pos_list):
  if(save.count(pos) == 0):
    save.append(pos)
    sentence_examples.append(sen_list[idx])
    part_of_speech.append(pos_list[idx])

In [ ]:
sentence_examples[0]

['I', 'am', 'a', 'boy']

In [ ]:
part_of_speech[0]

['PRON', 'VERB', 'DET', 'NOUN']

기존데이터에 문장 목록들 모두 넣어보고 가장 큰확률변화 일으키는 문장 파악

In [ ]:
## user의 베이스 데이터 축적
userdata = pd.read_csv('data/user.csv')
userdata

,Unnamed: 0,user,code,word,pos,time,correct,days,split,token_count,pos_set,poss,sen_pos,sen_correct
0,0,XEinXf5+,DRihrVmh0101,I,PRON,9.0,0,0.003,0,4,0-PRON-VERB,PRONVERBDETNOUN-PRON,PRONVERBDETNOUN,1
1,1,XEinXf5+,DRihrVmh0102,am,VERB,9.0,1,0.003,0,4,PRON-VERB-DET,PRONVERBDETNOUN-VERB,PRONVERBDETNOUN,1
2,2,XEinXf5+,DRihrVmh0103,a,DET,9.0,1,0.003,0,4,VERB-DET-NOUN,PRONVERBDETNOUN-DET,PRONVERBDETNOUN,1
3,3,XEinXf5+,DRihrVmh0104,boy,NOUN,9.0,0,0.003,0,4,DET-NOUN-0,PRONVERBDETNOUN-NOUN,PRONVERBDETNOUN,1
4,4,XEinXf5+,TOeLHxLS0101,I,PRON,12.0,1,0.005,1,4,0-PRON-VERB,PRONVERBADPPROPN-PRON,PRONVERBADPPROPN,1
5,5,XEinXf5+,TOeLHxLS0102,am,VERB,12.0,1,0.005,1,4,PRON-VERB-ADP,PRONVERBADPPROPN-VERB,PRONVERBADPPROPN,1
6,6,XEinXf5+,TOeLHxLS0103,from,ADP,12.0,1,0.005,1,4,VERB-ADP-PROPN,PRONVERBADPPROPN-ADP,PRONVERBADPPROPN,1
7,7,XEinXf5+,TOeLHxLS0104,Mexico,PROPN,12.0,0,0.005,1,4,ADP-PROPN-0,PRONVERBADPPROPN-PROPN,PRONVERBADPPROPN,1


## 추천 모듈화

In [ ]:
for rcount in range(5):
  print(rcount+1,"번째 추천")
  rand_idx = random.sample(range(len(part_of_speech)), 10)
  sentence_examples_10 = []
  part_of_speech_10 = []
  for i in rand_idx:
    sentence_examples_10.append(sentence_examples[i])
    part_of_speech_10.append(part_of_speech[i])
  ## 모든 문장 한번씩 넣어보기 ## 3분소요
  max_effect_prob = 0
  if(rcount == 0):
    for i in range(10): #len(sentence_examples)
      ## userdata의 평균확률
      userdata_list = userdata.values.tolist()
      userseq = import_file.DKTDef.test(100, userdata_list)
      orig_prob = import_file.DKTDef.inference(userseq)

      if(i==0):
        print("현재성취도: ", orig_prob)

      for j in range(len(sentence_examples_10[i])):
        input_sentence = {'word': sentence_examples_10[i][j], 'pos': part_of_speech_10[i][j], 'correct': 1, 'sen_pos': "".join(part_of_speech_10[i]), 'sen_correct': 1}
        if(j > 0):
          recomdata = recomdata.append(input_sentence, ignore_index=True)
        else:
          recomdata = userdata.append(input_sentence, ignore_index=True)
      
      ## recomdata의 평균확률
      recomdata = recomdata.values.tolist()
      recommend_seq = import_file.DKTDef.test(100, recomdata)
      recommend_prob = import_file.DKTDef.inference(recommend_seq)

      ## recomdata - userdata 차이
      effect_prob = recommend_prob - orig_prob
      if(effect_prob > max_effect_prob):
        max_effect_prob = effect_prob
        best_string = sentence_examples_10[i]
        best_string_pos = part_of_speech_10[i]

    from googletrans import Translator
    translator = Translator()
    a = translator.translate(' '.join(best_string), src='en', dest='ko')
    correct_list = []
    print("==아래문장을 작문하세요==")
    print("(예상 상승 성취도 : ", max_effect_prob, "%)")
    print(a.text)
    print("==단어 개수에 맞춰 하나씩 입력하세요==")
    for i in range(len(best_string)):
      print("[____]", end=' ')
    print()
    for str_ in best_string:
      wordinput = str(input())
      if(wordinput == str_):
        correct_list.append(1)
      else:
        correct_list.append(0)
    print("정답: ", ' '.join(best_string))
    print()

  else:
    max_effect_prob = 0
    for r in range(len(correct_list)):
        input_sentence = {'word': best_string[r], 'pos': best_string_pos[r], 'correct': correct_list[r], 'sen_pos': "".join(best_string_pos), 'sen_correct': 1}
        if(r > 0):
          userdata_new = userdata_new.append(input_sentence, ignore_index=True)
        else:
          userdata_new = userdata.append(input_sentence, ignore_index=True)

    for i in range(10): #len(sentence_examples)
      ## userdata + 추천문장의 평균확률
      userdata_list = userdata_new.values.tolist()
      userseq = import_file.DKTDef.test(100, userdata_list)
      prob = import_file.DKTDef.inference(userseq)
      if(i==0):
        print("현재성취도: ", prob)

      for j in range(len(sentence_examples_10[i])):
        input_sentence = {'word': sentence_examples_10[i][j], 'pos': part_of_speech_10[i][j], 'correct': 1, 'sen_pos': "".join(part_of_speech_10[i]), 'sen_correct': 1}
        if(j > 0):
          recomdata = recomdata.append(input_sentence, ignore_index=True)
        else:
          recomdata = userdata_new.append(input_sentence, ignore_index=True)

      ## recomdata의 평균확률
      recomdata = recomdata.values.tolist()
      recommend_seq = import_file.DKTDef.test(100, recomdata)
      recommend_prob = import_file.DKTDef.inference(recommend_seq)
      ## recomdata - userdata 차이
      effect_prob = recommend_prob - prob
      if(effect_prob > max_effect_prob):
        max_effect_prob = effect_prob
        best_string = sentence_examples_10[i]
        best_string_pos = part_of_speech_10[i]

    translator = Translator()
    a = translator.translate(' '.join(best_string), src='en', dest='ko')
    correct_list = []
    print("==아래문장을 작문하세요==")
    print("(예상 상승 성취도 : ", max_effect_prob, "%)")
    print(a.text)
    print("==단어 개수에 맞춰 하나씩 입력하세요==")
    for i in range(len(best_string)):
      print("[____]", end=' ')
    print()
    for str_ in best_string:
      wordinput = str(input())
      if(wordinput == str_):
        correct_list.append(1)
      else:
        correct_list.append(0)
    print("정답: ", ' '.join(best_string))
    print()

## 추천문제 풀이후 변화 성취도
print()
print("==추천문장 풀이후 성취도 변화==")
for r in range(len(correct_list)):
    input_sentence = {'word': best_string[r], 'pos': best_string_pos[r], 'correct': correct_list[r], 'sen_pos': "".join(best_string_pos), 'sen_correct': 1}
    if(r > 0):
      userdata_new = userdata_new.append(input_sentence, ignore_index=True)
    else:
      userdata_new = userdata_new.append(input_sentence, ignore_index=True)

for i in range(10): #len(sentence_examples)
  ## userdata + 추천문장의 평균확률
  userdata_list = userdata_new.values.tolist()
  userseq = import_file.DKTDef.test(100, userdata_list)
  prob = import_file.DKTDef.inference(userseq)

print("이전 성취도: ", orig_prob)
print("변화 성취도: ", prob)

1 번째 추천
현재성취도:  61.60551100037992
==아래문장을 작문하세요==
(예상 상승 성취도 :  7.4785875622183084 %)
나는 읽을 시간이 없다
==단어 개수에 맞춰 하나씩 입력하세요==
[____] [____] [____] [____] [____] [____] [____] [____] 
I
don't
have
time
to
read
the
book
정답:  I do not have time for the reading

2 번째 추천
현재성취도:  55.623493529856205
==아래문장을 작문하세요==
(예상 상승 성취도 :  10.701532941311598 %)
나는 언니에게 편지를 쓰고 싶었다
==단어 개수에 맞춰 하나씩 입력하세요==
[____] [____] [____] [____] [____] [____] [____] [____] 
I
want
to
write
the
letter
for
sister
정답:  I have wanted to write to my sister

3 번째 추천
현재성취도:  61.09842327423394
==아래문장을 작문하세요==
(예상 상승 성취도 :  5.79591472633183 %)
나는 큰 보트를 타본 적이 없습니다
==단어 개수에 맞춰 하나씩 입력하세요==
[____] [____] [____] [____] [____] [____] [____] [____] 
I
have
never
been
on
a
big
boat
정답:  I have never been on a large boat

4 번째 추천
현재성취도:  66.2017211318016
==아래문장을 작문하세요==
(예상 상승 성취도 :  4.022879106923938 %)
눈에서 멀어지면 마음에서도 멀어진 다
==단어 개수에 맞춰 하나씩 입력하세요==
[____] [____] [____] [____] [____] [____] 
Out
of
sight,
out
of
mind
정답:  Out of sight out 